In [2]:
import torch
from gensim.models.keyedvectors import KeyedVectors
import os
import pandas as pd
import numpy as np
import texthero as hero
import sklearn
from sklearn.preprocessing import LabelEncoder
from typing import Union
from texthero import preprocessing


filepath_bin = '../Data/Chapter7/GoogleNews-vectors-negative300.bin'
filedir_text = '../Data/Output/Chapter6'
filedir_out ='../Data/Output/Chapter8'
train_path = os.path.join(filedir_text, "ex50-train.txt")
test_path = os.path.join(filedir_text, "ex50-test.txt")
val_path = os.path.join(filedir_text, "ex50-val.txt")
word_vector = KeyedVectors.load_word2vec_format(filepath_bin, binary=True)

def preprocess( df: pd.DataFrame) -> pd.DataFrame:
    pipelines = [
        preprocessing.fillna,
        preprocessing.lowercase,
        preprocessing.remove_digits,
        preprocessing.remove_punctuation,
        preprocessing.remove_stopwords,
        preprocessing.remove_diacritics
    ]
    return hero.clean(df,pipelines)

def make_tensor( titles: list) -> torch.tensor:
    X = torch.zeros(len(titles), 300, dtype=torch.float)
    for i, title in enumerate(titles):
        words_title = title.split(' ')
        skipped_cnt = 0
        for word in words_title:
            if word not in word_vector:
                skipped_cnt += 1
                continue
            X[i] += word_vector[word]
        X /= max(len(words_title) - skipped_cnt, 1)  # avoid zero division
    return X

def make_article_vectors(df: pd.DataFrame) -> torch.tensor:
    df['clean_title'] = preprocess(df)
    titles = df['clean_title'].tolist()
    return make_tensor(titles)

def label_fit_transform(df: pd.DataFrame) -> None:
    le = LabelEncoder()
    le.fit(df['category'])
    return le.transform(df['category'])





# # ==========MAKE X==========


def represent_articles_as_average_of_words(df,file_out ):
    x = make_article_vectors(df['title'])
    path_out = os.path.join(filedir_out, file_out)
    torch.save(x, path_out)
    return x

df_train = pd.read_csv(train_path,sep='\t')
df_test = pd.read_csv(test_path,'\t')
df_val = pd.read_csv(val_path,sep='\t')

x_train_torch = represent_articles_as_average_of_words(df_train, file_out="ex70-train.pt")
x_test_torch = represent_articles_as_average_of_words(df_test, file_out="ex70-test.pt")
x_val_torch = represent_articles_as_average_of_words(df_val, file_out="ex70-val.pt")





# ==========MAKE Y==========

def create_y_labels(df, file_out):
    y = label_fit_transform(df)
    y_torch = torch.tensor(y)
    path_out = os.path.join(filedir_out,file_out)
    torch.save(y_torch, path_out)
    return y_torch

y_train_torch = create_y_labels(df_train, file_out = "ex70-train_label.pt")
y_val_torch = create_y_labels(df_val, file_out = "ex70-val_label.pt")
y_test_torch = create_y_labels(df_test, file_out = "ex70-test_label.pt")

print(y_train_torch)
print(y_train_torch.shape)
print(y_test_torch)
print(y_test_torch.shape)



C:\Users\Admin\AppData\Local\Temp\ipykernel_33748\4245021261.py:69: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df_test = pd.read_csv(test_path,'\t')
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\texthero\preprocessing.py:105: FutureWarning: The default value of regex will change from True to False in a future version.
  return s.str.replace(pattern, symbols)
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\texthero\preprocessing.py:173: FutureWarning: The default value of regex will change from True to False in a future version.
  return s.str.replace(rf"([{string.punctuation}])+", symbol)
C:\Users\Admin\AppData\Local\Temp\ipykernel_33748\4245021261.py:46: SettingWithCopyWarning: 
A value is trying to be set on

tensor([0, 2, 1,  ..., 1, 0, 0], dtype=torch.int32)
torch.Size([10672])
tensor([3, 3, 2,  ..., 0, 1, 0], dtype=torch.int32)
torch.Size([1334])
